#  Итоговый файл по ссылке
- https://docs.google.com/spreadsheets/d/18Ckya7-63RrjakdjeirR4xkjCMZUsUuJ/edit?usp=sharing&ouid=100993897678532417326&rtpof=true&sd=true

# Знакомимся с данными

In [1]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl.styles import PatternFill

In [2]:
# Загрузка данных
excel_file = '../Задание №1.xlsx'
df_main = pd.read_excel(excel_file, sheet_name='Sheet2')
df_sheet1 = pd.read_excel(excel_file, sheet_name='Sheet1')

In [3]:
# Список новых имён для столбцов
new_column_names = ['Менеджер', 'Оператор', 'Предоставление реестров', '% предоставления', '"ДА", если предоставление 50% и более процентов', 'GUM', 'CNF', 'SV', 'розовая заливка']

# Создание словаря для переименования
rename_dict = dict(zip(df_main.columns, new_column_names))

# Переименование столбцов
df_main = df_main.rename(columns=rename_dict)

In [4]:
# Выведем названия столбцов, чтобы увидеть правильное написание
df_main.sample(5)

,Менеджер,Оператор,Предоставление реестров,% предоставления,"""ДА"", если предоставление 50% и более процентов",GUM,CNF,SV
10,Леденев Андрей,Орлова Таня,1.0,NaN,NaN,NaN,NaN,NaN
34,Брагарь Олег,Федорова Таня,0.0,NaN,NaN,NaN,NaN,NaN
74,Шлюшенков Александр,Бодрова Анна,1.0,NaN,NaN,NaN,NaN,NaN
62,Моргунов Андрей,Бодрова Анна,1.0,NaN,NaN,NaN,NaN,NaN
26,Малышев Сергей,Федорова Таня,1.0,NaN,NaN,NaN,NaN,NaN


#### #1
## Количество всех менеджеров по всем операторам

In [5]:
# 1. Количество всех менеджеров по всем операторам
total_managers = df_main['Менеджер'].nunique()
print(f"Всего менеджеров: {total_managers}")

Всего менеджеров: 64


#### #2
## Количество менеджеров с предоставленным реестром

In [6]:
# 2. Количество менеджеров с предоставленным реестром
managers_with_registry = df_main[df_main['Предоставление реестров'] >= 1]['Менеджер'].nunique()
print(f"Менеджеров с реестром: {managers_with_registry}")

Менеджеров с реестром: 37


#### #3
## Расчет процента выполнения по предоставлению реестров для каждого оператора

In [7]:
# 3. Расчет процента выполнения по предоставлению реестров для каждого оператора
operator_stats = df_main.groupby('Оператор').agg(
    total_managers=('Менеджер', 'nunique'),
    registry_provided=('Предоставление реестров', lambda x: sum(x >= 1))
)
operator_stats['% предоставления'] = (operator_stats['registry_provided'] / operator_stats['total_managers']) * 100

In [8]:
operator_stats

,total_managers,registry_provided,% предоставления
Оператор,,,
Бодрова Анна,20,13,65.000000
Орлова Таня,17,7,41.176471
Уткина Анна,9,9,100.000000
Федорова Таня,18,12,66.666667


### #4 #5 #6
## Заполняем колонки по условиям, агрегирует данные

In [9]:
operator_stats_df = operator_stats.reset_index()
# Создание словаря маппинга: значение из первой колонки -> значение из третьей колонки
mapping = dict(zip(operator_stats_df.iloc[:, 0], operator_stats_df.iloc[:, 3]))

# Заполнение колонки b в первой таблице
df_main['% предоставления'] = df_main['Оператор'].map(mapping)


In [10]:
# 4-5. Создание колонок с условиями
# df_main['розовая заливка'] = df_main['% предоставления'] >= 80
df_main[' "ДА", если предоставление 50% и более процентов'] = np.where(df_main['% предоставления'] >= 50, 'Да', '')

In [11]:
# 6. Агрегация данных из Sheet1 по менеджерам
product_summary = df_sheet1.groupby('Менеджер').agg({
    'GUM': 'sum',
    'CNF': 'sum',
    'SV': 'sum'
})

In [12]:
# Форматирование чисел (одна цифра после запятой с разделителем тысяч)
for col in ['GUM', 'CNF', 'SV']:
    product_summary[col] = product_summary[col].apply(lambda x: f"{x:,.1f}")

In [13]:
product_summary.sample(4)

,GUM,CNF,SV
Менеджер,,,
Магомедов Салман,"87,392.7","59,364.4","9,406.7"
Ларионов Роман,"38,542.9","26,287.4","4,168.7"
Алексеев Павел,"3,749.3","2,582.5",418.0
Суриков Владимир,"62,967.8","42,825.9","6,787.7"


##    Экспортируем

In [14]:
# Объединение данных
df = pd.merge(df_main, product_summary, on='Менеджер', how='left')
df = df.loc[:, ~df.columns.str.endswith('_x')]

# Переименовываем столбцы с суффиксом '_y', убирая '_y'
df.columns = df.columns.str.replace('_y', '', regex=False)
df.drop('"ДА", если предоставление 50% и более процентов', axis=1, inplace=True)


In [16]:
# Сохранение результатов
df.to_excel('h0gggg.xlsx', index=False)